## ANÁLISE DE UMA TORRE DE LT SUBMETIDA À AÇÃO DO VENTO

### MONTAGEM DA MATRIZ DE RIGIDEZ GLOBAL

Primeiro, estou fazendo um teste para um exemplo didático de treliça 3D. Se funcionar, usamos para a torre de LT.

In [154]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [155]:
# Lendo um arquivo em .xls que contém o número dos nós e suas coordenadas x, y e z

nos = pd.read_excel (r'data\nos.xlsx')  # Importa a planilha com as coordenadas dos nós
dados = nos.values                      # Transforma os dados da planilha em uma matriz

x = dados[:,1]
y = dados[:,2]
z = dados[:,3]

cone = pd.read_excel (r'data\elementos.xlsx')
dados2 = cone.values

nele = len(dados2)  # Número de elementos (barras)
nnos = len(dados)   # Número de Nós

ni = dados2[:,1]    # Nós iniciais
nf = dados2[:,2]    # Nós finais
A  = dados2[:,3]    # Área de seção em m²
E  = dados2[:,4]    # Módulo de Elasticidade em N/m²
EA = A*E            # Rigidez Axial

L = np.zeros(nele)  # Vetor que armazenará os comprimentos

Cx = np.zeros(nele) # Vetores que armazenarão os cossenos diretores
Cy = np.zeros(nele)
Cz = np.zeros(nele)

for i in range(nele):

    ni1 = ni[i] 
    nf1 = nf[i]
 
    L[i] = np.sqrt((x[int(ni1-1)]-x[int(nf1-1)])**2 + (y[int(ni1-1)]-y[int(nf1-1)])**2 + (z[int(ni1-1)]-z[int(nf1-1)])**2)

    Cx[i] = (x[int(ni1-1)]-x[int(nf1-1)])/L[i]
    Cy[i] = (y[int(ni1-1)]-y[int(nf1-1)])/L[i]
    Cz[i] = (z[int(ni1-1)]-z[int(nf1-1)])/L[i]

In [156]:
KG  =  np.zeros((3*nnos,3*nnos))  # Matriz global com zeros

# Valores que se repetem ao longo das matrizes de elementos:

k11 = Cx*Cx*EA/L;
k12 = Cx*Cy*EA/L;
k13 = Cx*Cz*EA/L;
k22 = Cy*Cy*EA/L;
k23 = Cy*Cz*EA/L;
k33 = Cz*Cz*EA/L;

for k in range(nnos-1):
    
    for p in range(nnos-1):
        i1   = 3*k;       #0
        i2   = 3*k + 1;   #1
        i3   = 3*k + 2;   #2
        i4   = 3*k + 3;   #3
        i5   = 3*k + 4;   #4
        i6   = 3*k + 5;   #5
    
        k11k = k11[p];
        k12k = k12[p];
        k13k = k13[p];
        k22k = k22[p];
        k23k = k23[p];
        k33k = k33[p];
       
        KG[i1,i1] = KG[i1,i1] + k11k;
        KG[i1,i2] = KG[i1,i2] + k12k;    KG[i2,i1] = KG[i2,i1] + k12k;
        KG[i1,i3] = KG[i1,i3] + k13k;    KG[i3,i1] = KG[i3,i1] + k13k; 
        KG[i1,i4] = KG[i1,i4] - k11k;    KG[i4,i1] = KG[i4,i1] - k11k; 
        KG[i1,i5] = KG[i1,i5] - k12k;    KG[i5,i1] = KG[i5,i1] - k12k;
        KG[i1,i6] = KG[i1,i6] - k13k;    KG[i6,i1] = KG[i6,i1] - k13k
      
            
        KG[i2,i2] = KG[i2,i2] + k22k;
        KG[i2,i3] = KG[i2,i3] + k23k;    KG[i3,i2] = KG[i3,i2] + k23k;
        KG[i2,i4] = KG[i2,i4] - k12k;    KG[i4,i2] = KG[i4,i2] + k12k;
        KG[i2,i5] = KG[i2,i5] - k22k;    KG[i5,i2] = KG[i5,i2] - k22k;
        KG[i2,i6] = KG[i2,i6] - k23k;    KG[i6,i2] = KG[i6,i2] - k23k;
        
    
        KG[i3,i3] = KG[i3,i3] + k33k;    
        KG[i3,i4] = KG[i3,i4] - k13k;    KG[i4,i3] = KG[i4,i3] - k13k;
        KG[i3,i5] = KG[i3,i5] - k23k;    KG[i5,i3] = KG[i5,i3] - k23k;
        KG[i3,i6] = KG[i3,i6] - k33k;    KG[i6,i3] = KG[i6,i3] - k33k;
        
       
        KG[i4,i4] = KG[i4,i4] + k11k;
        KG[i4,i5] = KG[i4,i5] + k12k;    KG[i5,i4] = KG[i5,i4] + k12k;
        KG[i4,i6] = KG[i4,i6] + k13k;    KG[i6,i4] = KG[i6,i4] + k13k;
       
        
        KG[i5,i5] = KG[i5,i5] + k22k;
        KG[i5,i6] = KG[i5,i6] + k23k;    KG[i6,i5] = KG[i6,i5] + k23k;
        
        KG[i6,i6] = KG[i6,i6] + k33k

print(KG)

[[ 1.36424185e+11  1.57051086e+05 -9.53901425e+05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.57051086e+05  2.18463552e+11  2.49102202e+06 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-9.53901425e+05  2.49102202e+06  1.38453706e+11 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.36424185e+11
   1.57051086e+05 -9.53901425e+05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.57051086e+05
   2.18463552e+11  2.49102202e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -9.53901425e+05
   2.49102202e+06  1.38453706e+11]]


In [147]:
#plt.figure(2, figsize=(8,8))
#KG = plt.imshow(KG, cmap=plt.cm.bone)

In [148]:
#Matriz de Massa Consistente

rho = 7850  # Massa específica em kg/m³

MG  =  np.zeros((3*nnos,3*nnos))  # Matriz de Massa global com zeros

# Valores que se repetem ao longo das matrizes de elementos:

m11 = 2*rho*A*L/6;
m14 = 1*rho*A*L/6;

for k in range(nnos-1):
    
    for p in range(nnos-1):
        i1   = 3*k;       #0
        i2   = 3*k + 1;   #1
        i3   = 3*k + 2;   #2
        i4   = 3*k + 3;   #3
        i5   = 3*k + 4;   #4
        i6   = 3*k + 5;   #5
    
        m11m = m11[p];
        m14m = m14[p];
       
        MG[i1,i1] = MG[i1,i1] + m11m;
        MG[i1,i2] = MG[i1,i2] + 0;       MG[i2,i1] = MG[i2,i1] + 0;
        MG[i1,i3] = MG[i1,i3] + 0;       MG[i3,i1] = MG[i3,i1] + 0; 
        MG[i1,i4] = MG[i1,i4] + m14m;    MG[i4,i1] = MG[i4,i1] + m14m; 
        MG[i1,i5] = MG[i1,i5] + 0;       MG[i5,i1] = MG[i5,i1] + 0;
        MG[i1,i6] = MG[i1,i6] + 0;       MG[i6,i1] = MG[i6,i1] + 0
      
            
        MG[i2,i2] = MG[i2,i2] + m11m;
        MG[i2,i3] = MG[i2,i3] + 0;       MG[i3,i2] = MG[i3,i2] + 0;
        MG[i2,i4] = MG[i2,i4] + 0;       MG[i4,i2] = MG[i4,i2] + 0;
        MG[i2,i5] = MG[i2,i5] + m14m;    MG[i5,i2] = MG[i5,i2] + m14m;
        MG[i2,i6] = MG[i2,i6] + 0;       MG[i6,i2] = MG[i6,i2] + 0;
        
    
        MG[i3,i3] = MG[i3,i3] + m11m;    
        MG[i3,i4] = MG[i3,i4] + 0;       MG[i4,i3] = MG[i4,i3] + 0;
        MG[i3,i5] = MG[i3,i5] + 0;       MG[i5,i3] = MG[i5,i3] + 0;
        MG[i3,i6] = MG[i3,i6] + m14m;    MG[i6,i3] = MG[i6,i3] + m14m;
        
       
        MG[i4,i4] = MG[i4,i4] + m11m;
        MG[i4,i5] = MG[i4,i5] + 0;      MG[i5,i4] = MG[i5,i4] + 0;
        MG[i4,i6] = MG[i4,i6] + 0;      MG[i6,i4] = MG[i6,i4] + 0;
       
        
        MG[i5,i5] = MG[i5,i5] + m11m;
        MG[i5,i6] = MG[i5,i6] + 0;      MG[i6,i5] = MG[i6,i5] + 0;
        
        MG[i6,i6] = MG[i6,i6] + m11m

print(MG)

[[71687.02065941     0.             0.         ...     0.
      0.             0.        ]
 [    0.         71687.02065941     0.         ...     0.
      0.             0.        ]
 [    0.             0.         71687.02065941 ...     0.
      0.             0.        ]
 ...
 [    0.             0.             0.         ... 71687.02065941
      0.             0.        ]
 [    0.             0.             0.         ...     0.
  71687.02065941     0.        ]
 [    0.             0.             0.         ...     0.
      0.         71687.02065941]]


In [149]:
#Eliminando das matrizes os graus de liberdade restringidos

KG = KG[12:,12:]
MG = MG[12:,12:]
#print(KG)
len(MG)

756

In [150]:
# Solução do problema de autovalores com o módulo scipy

import scipy.linalg as sc
w2, Phi  =  sc.eig(KG, MG)

# Impõe ordem crescente dos modos
iw  = w2.argsort()
w2  = w2[iw]
Phi = Phi[:,iw]

# Converte autovalores em frequências
wk  = np.sqrt(np.real(w2)) 
fk  = wk/2/np.pi

print(fk[0:9])

[0.79014162 0.79599726 0.99988209 2.37045567 2.3880227  2.99968502
 3.95086181 3.98014092 4.99960453]
